<a href="https://colab.research.google.com/github/rohithreddy999/Transfer_learning/blob/main/AUTOMATED_WEATHER_CLASSIFICATION_USING_TRANSFER_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/AUTOMATED_WEATHER_CLASSIFICATION_USING_TRANSFER_LEARNING/archive.zip"

Archive:  /content/drive/MyDrive/AUTOMATED_WEATHER_CLASSIFICATION_USING_TRANSFER_LEARNING/archive.zip
  inflating: Multi-class Weather Dataset/Cloudy/cloudy1.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy10.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy100.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy101.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy102.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy103.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy104.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy105.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy106.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy107.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy108.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy109.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy11.jpg  
  inflating: Multi-class Weather Dataset/Cloudy/cloudy110.jpg  
  infl

In [3]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [4]:
base_dir="/content/Multi-class Weather Dataset"

In [5]:
train_gen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train = train_gen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size= 32,
    class_mode='categorical',
    subset='training')

validation_gen = train_gen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size= 32,
    class_mode='categorical',
    subset='validation')

Found 901 images belonging to 4 classes.
Found 224 images belonging to 4 classes.


VGG16

In [6]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [7]:
# Adding the preprocessing layer to the front of vgg
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
# Train model with existing weights
for layer in vgg.layers:
  print(layer)

In [11]:
# Train model with existing weights
for layer in vgg.layers:
  layer.trainable=False

In [12]:
x = Flatten()(vgg.output)

In [13]:
# output layer
prediction = Dense(4,activation='softmax')(x)
# Create Vgg16 model
model = Model(inputs=vgg.input,outputs=prediction)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [16]:
model.fit_generator(train,validation_data=validation_gen,epochs=10,steps_per_epoch=len(train),
                    validation_steps=len(validation_gen))

<ipython-input-16-3b4b868b8d22>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=validation_gen,epochs=10,steps_per_epoch=len(train),


Epoch 1/10
29/29 [==============================] - 35s 764ms/step - loss: 0.9922 - accuracy: 0.6471 - val_loss: 1.1540 - val_accuracy: 0.6473
Epoch 2/10
29/29 [==============================] - 26s 904ms/step - loss: 0.3346 - accuracy: 0.8746 - val_loss: 0.6467 - val_accuracy: 0.7902
Epoch 3/10
29/29 [==============================] - 21s 710ms/step - loss: 0.1826 - accuracy: 0.9445 - val_loss: 0.4057 - val_accuracy: 0.8839
Epoch 4/10
29/29 [==============================] - 20s 690ms/step - loss: 0.1305 - accuracy: 0.9689 - val_loss: 0.4452 - val_accuracy: 0.8571
Epoch 5/10
29/29 [==============================] - 20s 688ms/step - loss: 0.0989 - accuracy: 0.9811 - val_loss: 0.3957 - val_accuracy: 0.8705
Epoch 6/10
29/29 [==============================] - 21s 710ms/step - loss: 0.1014 - accuracy: 0.9778 - val_loss: 0.3264 - val_accuracy: 0.8795
Epoch 7/10
29/29 [==============================] - 21s 708ms/step - loss: 0.0893 - accuracy: 0.9811 - val_loss: 0.4530 - val_accuracy: 0.8304

In [17]:
model.save("Automated_weather_classification.h5")

RESNET50

In [18]:
#RESNET 50
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 3s 0us/step


In [19]:
for layer in resnet.layers:
  layer.trainable=False

In [20]:
for layer in resnet.layers:
  print(layer)

In [21]:
x = Flatten()(resnet.output)

In [22]:
out = Dense(4, activation='softmax')(x)

In [23]:
res_model = Model(inputs=resnet.input,outputs=out)

In [24]:
res_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [25]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
res_model.fit(train,epochs=5,validation_data=validation_gen,steps_per_epoch=len(train),
              validation_steps=len(validation_gen))

Epoch 1/5
29/29 [==============================] - 27s 772ms/step - loss: 4.5890 - accuracy: 0.2963 - val_loss: 1.4691 - val_accuracy: 0.3616
Epoch 2/5
29/29 [==============================] - 20s 697ms/step - loss: 1.4455 - accuracy: 0.3685 - val_loss: 1.3120 - val_accuracy: 0.3304
Epoch 3/5
29/29 [==============================] - 20s 697ms/step - loss: 1.2164 - accuracy: 0.5017 - val_loss: 1.0394 - val_accuracy: 0.5536
Epoch 4/5
29/29 [==============================] - 21s 743ms/step - loss: 1.1845 - accuracy: 0.5305 - val_loss: 1.3555 - val_accuracy: 0.4062
Epoch 5/5
29/29 [==============================] - 21s 726ms/step - loss: 1.1225 - accuracy: 0.5361 - val_loss: 1.1372 - val_accuracy: 0.4866


INCEPTIONV3

In [27]:
#inseption-v3
from tensorflow.keras.applications.inception_v3 import InceptionV3
inc = InceptionV3(include_top=False,weights="imagenet",input_shape=(224, 224, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [28]:
for layer in inc.layers:
  layer.trainable=False

In [29]:
for layer in inc.layers:
  print(layer)

In [30]:
x1= Flatten()(inc.output)

In [31]:
pred = Dense(4, activation='softmax')(x1)

In [32]:
inception_model = Model(inputs=inc.input,outputs=pred)

In [33]:
inception_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [34]:
inception_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
inception_model.fit(train,epochs=5,validation_data=validation_gen,steps_per_epoch=len(train),
              validation_steps=len(validation_gen))

Epoch 1/5
29/29 [==============================] - 28s 763ms/step - loss: 3.1814 - accuracy: 0.6948 - val_loss: 1.0506 - val_accuracy: 0.8616
Epoch 2/5
29/29 [==============================] - 20s 679ms/step - loss: 0.5123 - accuracy: 0.9201 - val_loss: 0.7987 - val_accuracy: 0.8616
Epoch 3/5
29/29 [==============================] - 20s 669ms/step - loss: 0.1471 - accuracy: 0.9656 - val_loss: 1.1385 - val_accuracy: 0.8125
Epoch 4/5
29/29 [==============================] - 20s 689ms/step - loss: 0.2070 - accuracy: 0.9534 - val_loss: 0.9986 - val_accuracy: 0.8348
Epoch 5/5
29/29 [==============================] - 20s 690ms/step - loss: 0.1463 - accuracy: 0.9634 - val_loss: 0.5372 - val_accuracy: 0.8884


XCEPTION

In [130]:
from keras.applications.xception import Xception

In [132]:
xception = Xception(include_top=False,input_shape=(224,224,3))

In [133]:
for layer in xception.layers:
  layer.trainable=False

In [134]:
for layer in xception.layers:
  print(layer)


In [135]:
x = Flatten()(xception.output)

In [136]:
predic = Dense(4, activation='softmax')(x)

In [137]:
xce_model = Model(inputs=xception.input,outputs=predic)

In [138]:

xce_model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [143]:

xce_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [144]:
xce_model.fit(train,epochs=5,validation_data=validation_gen,steps_per_epoch=len(train),
              validation_steps=len(validation_gen))

Epoch 1/5
29/29 [==============================] - 25s 783ms/step - loss: 0.9935 - accuracy: 0.7858 - val_loss: 0.9955 - val_accuracy: 0.7902
Epoch 2/5
29/29 [==============================] - 21s 730ms/step - loss: 0.3842 - accuracy: 0.9134 - val_loss: 0.8885 - val_accuracy: 0.8304
Epoch 3/5
29/29 [==============================] - 21s 725ms/step - loss: 0.4068 - accuracy: 0.9267 - val_loss: 0.7422 - val_accuracy: 0.8750
Epoch 4/5
29/29 [==============================] - 21s 701ms/step - loss: 0.2901 - accuracy: 0.9412 - val_loss: 0.9252 - val_accuracy: 0.8661
Epoch 5/5
29/29 [==============================] - 20s 678ms/step - loss: 0.1218 - accuracy: 0.9656 - val_loss: 0.6201 - val_accuracy: 0.8795
